In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv("upmo_dataset_1.csv")

In [ ]:
print(df.head())
print(df.columns)

In [ ]:
# Identify identifier columns (first three) and value columns (monthly dates)
id_cols = ['CATEGORY', 'ARTICLE', 'DESCRIPTION']
date_cols = df.columns.difference(id_cols)

# Melt the dataframe
df_long = pd.melt(df, 
                  id_vars=id_cols, 
                  value_vars=date_cols, 
                  var_name='DATE', 
                  value_name='SALES')

In [ ]:
# Add full date
df_long['DATE'] = pd.to_datetime(df_long['DATE'], format='%y-%b')

In [ ]:
# Convert sales to numeric
df_long['SALES'] = pd.to_numeric(df_long['SALES'], errors='coerce')

# Drop missing or invalid values
df_long = df_long.dropna(subset=['SALES'])

In [ ]:
df_long = df_long.sort_values(by=['CATEGORY', 'ARTICLE', 'DATE'])

In [ ]:
df_long['YEAR'] = df_long['DATE'].dt.year
df_long['MONTH'] = df_long['DATE'].dt.month
df_long['QUARTER'] = df_long['DATE'].dt.quarter
df_long['MONTH_NAME'] = df_long['DATE'].dt.month_name()

# Define Philippine holiday season months
philippine_holiday_months = [9, 10, 11, 12]

# Tag if the sale was during holiday season
df_long['IS_HOLIDAY_SEASON'] = df_long['MONTH'].isin(philippine_holiday_months).astype(int)

In [ ]:
df_long.to_csv("cleaned_sales_data.csv", index=False)
# df_long
# df_long.columns()

### What Do We Want from the Data? Define Purpose.

#### Here are some example.

| **Goal**                         | **Machine Learning Type**           | **Example**                              |
|----------------------------------|-------------------------------------|------------------------------------------|
| Forecast future sales            | Time Series Forecasting             | Predict Jan 2024 sales                   |
| Classify demand trend            | Classification                      | Will demand increase/decrease?          |
| Detect anomalies                 | Anomaly Detection                   | Flag sudden drops or spikes             |
| Cluster similar items            | Clustering                          | Group articles with similar sales patterns |
| Analyze seasonality or trends    | Unsupervised / Time Series Decomposition | Detect monthly/seasonal patterns     |

### Testing Goal 1 using Prophet - Facebook meta model

In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import os
import re
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

warnings.filterwarnings('ignore')

# === PARAMETERS ===
forecast_periods = 6  # Change to 3, 6, or 12 months ahead
forecast_start_month = '2024-01'
forecast_df = []
forecast_plot_dir = "prophet_forecasts"

# === LOAD DATA ===
df = pd.read_csv("cleaned_sales_data.csv", parse_dates=['DATE'])

# Create plot directory
os.makedirs(forecast_plot_dir, exist_ok=True)

# === FORECAST LOOP ===
for (cat, art, desc), group in df.groupby(['CATEGORY', 'ARTICLE', 'DESCRIPTION']):
    print(f"Processing: {cat} | {art} | {desc}")

    ts = group[['DATE', 'SALES']].rename(columns={'DATE': 'ds', 'SALES': 'y'}).sort_values('ds')

    # Check data sufficiency
    if ts.shape[0] < 24 or ts['y'].isnull().any():
        print("❌ Skipped: not enough data or contains missing values.")
        continue

    # Fit Prophet
    try:
        model = Prophet(yearly_seasonality=True)
        model.fit(ts)
    except Exception as e:
        print(f"❌ Prophet failed: {e}")
        continue

    # Forecast
    future = model.make_future_dataframe(periods=forecast_periods, freq='MS')
    forecast = model.predict(future)

    # Historical metrics (last 3 actual vs predicted)
    try:
        if ts.shape[0] >= 27:
            actual_3 = ts.set_index('ds').iloc[-3:]
            pred_3 = model.predict(ts).set_index('ds').loc[actual_3.index]['yhat']
            mae = mean_absolute_error(actual_3['y'], pred_3)
            rmse = mean_squared_error(actual_3['y'], pred_3, squared=False)
        else:
            mae = rmse = None
    except:
        mae = rmse = None

    # Last actual
    last_actual = ts['y'].iloc[-1]

    # Filter forecast results by date (e.g., Jan–Jun 2024)
    forecast_filtered = forecast[forecast['ds'] >= pd.to_datetime(forecast_start_month)]

    for _, row in forecast_filtered.iterrows():
        forecast_df.append({
            "CATEGORY": cat,
            "ARTICLE": art,
            "DESCRIPTION": desc,
            "LAST_ACTUAL_SALES": last_actual,
            "PREDICTED_DATE": row['ds'].date(),
            "PREDICTED_SALES": row['yhat'],
            "GROWTH": row['yhat'] - last_actual,
            "MAE": mae,
            "RMSE": rmse
        })

    # Save plot
    try:
        fig = model.plot(forecast)
        safe_name = f"{cat}_{art}_{desc[:30]}".replace(' ', '_').replace('/', '-')
        fig.savefig(f"{forecast_plot_dir}/{safe_name}.png")
        plt.close(fig)
    except:
        print("⚠️ Plot save failed.")

    print(f"✔ Done: {desc}")

# === SAVE RESULTS ===
forecast_df = pd.DataFrame(forecast_df)
forecast_df.to_csv("prophet_forecast_results.csv", index=False)

# === SUMMARY METRICS ===
summary = forecast_df.groupby(['CATEGORY', 'ARTICLE', 'DESCRIPTION'])[['MAE', 'RMSE']].mean().reset_index()
summary.to_csv("prophet_forecast_summary_metrics.csv", index=False)

# === PLOT TOP GROWTH ITEMS ===
print("📊 Plotting top growth items...")
jan_forecast = forecast_df[forecast_df['PREDICTED_DATE'].astype(str).str.startswith('2024-01')]
top5 = jan_forecast.sort_values(by='GROWTH', ascending=False).head(5)

plt.figure(figsize=(10, 6))
plt.barh(top5['DESCRIPTION'], top5['GROWTH'], color='green')
plt.xlabel("Growth (Predicted Jan 2024 - Last Sales)")
plt.title("Top 5 Items with Highest Forecasted Growth (Jan 2024)")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig("top5_forecast_growth.png")
plt.show()

### Scikit-learn

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# === CONFIG ===
forecast_horizon = 12
min_required_rows = 24
plot_dir = "rf_forecast_plots"
os.makedirs(plot_dir, exist_ok=True)

# === LOAD DATA ===
df = pd.read_csv("cleaned_sales_data.csv", parse_dates=["DATE"])

# === FEATURE ENGINEERING FUNCTION ===
def create_features(data, lags=12):
    df_feat = data.copy()
    for lag in range(1, lags + 1):
        df_feat[f'lag_{lag}'] = df_feat['SALES'].shift(lag)
    df_feat['month'] = df_feat.index.month
    df_feat['quarter'] = df_feat.index.quarter
    df_feat = df_feat.dropna()
    return df_feat

# === FORECAST RESULTS ===
all_forecasts = []
summary_metrics = []

# === GROUP LOOP ===
for (cat, art, desc), group in df.groupby(["CATEGORY", "ARTICLE", "DESCRIPTION"]):
    print(f"Processing: {cat} | {art} | {desc}")
    df_item = group[['DATE', 'SALES']].copy()
    df_item = df_item.set_index('DATE').sort_index().asfreq('MS')

    if df_item.shape[0] < min_required_rows or df_item['SALES'].isnull().sum() > 0:
        print("❌ Skipped: not enough data or contains missing values.")
        continue

    # Feature engineering
    data_supervised = create_features(df_item)
    if data_supervised.shape[0] <= forecast_horizon:
        print("❌ Skipped: not enough supervised rows.")
        continue

    train = data_supervised.iloc[:-forecast_horizon]
    test = data_supervised.iloc[-forecast_horizon:]

    X_train = train.drop(columns='SALES')
    y_train = train['SALES']
    X_test = test.drop(columns='SALES')
    y_test = test['SALES']

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # Recursive forecasting
    last_known = data_supervised.iloc[-1]
    lag_values = [last_known[f'lag_{i}'] for i in range(1, 13)]
    current_date = df_item.index[-1]
    future_preds = []

    for step in range(forecast_horizon):
        current_date += pd.DateOffset(months=1)
        month = current_date.month
        quarter = (month - 1) // 3 + 1
        features = {f'lag_{i}': lag_values[i - 1] for i in range(1, 13)}
        features['month'] = month
        features['quarter'] = quarter

        X_pred = pd.DataFrame([features])
        pred = model.predict(X_pred)[0]

        future_preds.append({
            "CATEGORY": cat,
            "ARTICLE": art,
            "DESCRIPTION": desc,
            "PREDICTED_DATE": current_date,
            "PREDICTED_SALES": pred,
            "LAST_ACTUAL_SALES": df_item['SALES'].iloc[-1],
            "GROWTH": pred - df_item['SALES'].iloc[-1],
            "MAE": mae,
            "RMSE": rmse
        })

        lag_values = [pred] + lag_values[:-1]

    # Save plot
    try:
        future_df = pd.DataFrame(future_preds)
        plt.figure(figsize=(10, 4))
        plt.plot(df_item.index, df_item['SALES'], label='Historical')
        plt.plot(future_df['PREDICTED_DATE'], future_df['PREDICTED_SALES'], label='Forecast', color='orange')
        plt.title(f"{desc}")
        plt.legend()
        plt.tight_layout()
        filename = f"{cat}_{art}_{desc[:30].replace('/', '_').replace(' ', '_')}.png"
        plt.savefig(os.path.join(plot_dir, filename))
        plt.close()
    except:
        print("⚠️ Plot failed.")

    all_forecasts.extend(future_preds)
    summary_metrics.append({
        "CATEGORY": cat,
        "ARTICLE": art,
        "DESCRIPTION": desc,
        "MAE": mae,
        "RMSE": rmse
    })

    print(f"✔ Done: {desc}")

# === SAVE RESULTS ===
forecast_df = pd.DataFrame(all_forecasts)
forecast_df.to_csv("rf_forecast_all_items.csv", index=False)

summary_df = pd.DataFrame(summary_metrics).sort_values(by='RMSE')
summary_df.to_csv("rf_forecast_summary_metrics.csv", index=False)

print("✅ All forecasts completed.")

Processing: BUILDINGS | LUMBER | LUMBER 2" X 3" X 12'/14' KD
✔ Done: LUMBER 2" X 3" X 12'/14' KD
Processing: BUILDINGS | LUMBER | LUMBER 2" X 4" X 14' KD
✔ Done: LUMBER 2" X 4" X 14' KD
Processing: BUILDINGS | PLYWOOD | PLYWOOD 1/4"
✔ Done: PLYWOOD 1/4"
Processing: BUILDINGS | PLYWOOD | PLYWOOD 3/4"
✔ Done: PLYWOOD 3/4"
Processing: BUILDINGS | SCREEN MATTING | SCREEN MATTING
✔ Done: SCREEN MATTING
Processing: CUSTODIAL | BROOM  | BROOM (WALIS TAMBO)
✔ Done: BROOM (WALIS TAMBO)
Processing: CUSTODIAL | DISENFECTANT SPRAY | DISENFECTANT SPRAY Citrus Scent (340g)
✔ Done: DISENFECTANT SPRAY Citrus Scent (340g)
Processing: CUSTODIAL | DUTCH CLEANSER | DUTCH CLEANSER
✔ Done: DUTCH CLEANSER
Processing: ELECTRICAL | CONVINIENCE OUTLET | 2 GANG W/ GROUNDING PLATE PANASONIC DUPLEX
✔ Done: 2 GANG W/ GROUNDING PLATE PANASONIC DUPLEX
Processing: ELECTRICAL | FLOURESCENT TUBE | FLOURESCENT TUBE 40W
✔ Done: FLOURESCENT TUBE 40W
Processing: ELECTRICAL | HIGH SODIUM LAMP | HIGH SODIUM LAMP  250W
✔ Done: